In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
import tensorflow_hub as hub

In [3]:
random_state_split = 42
Dropout_num = 0
learning_rate = 5.95e-6
valid = 0.15
epochs_num = 3
batch_size_num = 16
target_corrected = False
target_big_corrected = False

In [4]:
url = '/content/drive/My Drive/SyncPC/Data Analytic/DisasterTweet'

train_df = pd.read_csv(f'{url}/train.csv')
test_df = pd.read_csv(f'{url}/test.csv')

In [5]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [6]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    
    if Dropout_num == 0:
        out = Dense(1, activation='sigmoid')(clf_output)
    else:
        x = Dropout(Dropout_num)(clf_output)
        out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [7]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [8]:
# !pip install bert-tensorflow

In [9]:
# Load tokenizer from the bert layer
from bert import tokenization

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [10]:
import sys
from absl import flags
sys.argv=['preserve_unused_tokens=False']
flags.FLAGS(sys.argv)

# Encode the text into tokens, masks, and segment flags
X_train = bert_encode(train_df.text.values, tokenizer, max_len=160)
X_test = bert_encode(test_df.text.values, tokenizer, max_len=160)
y_train = train_df.target.values

In [11]:
# Build BERT model with my tuning
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 160)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 160)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 160)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 1024),       335141889   ['input_word_ids[0][0]',         
                                 (None, 160, 1024)]               'input_mask[0][0]',         

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    X_train, y_train,
    validation_split = valid,
    epochs = epochs_num, # recomended 3-5 epochs
    callbacks=[checkpoint],
    batch_size = batch_size_num
)

Epoch 1/4
373/373 [==============================] - 832s 2s/step - loss: 0.4512 - accuracy: 0.8014 - val_loss: 0.4009 - val_accuracy: 0.8281
Epoch 2/4
373/373 [==============================] - 631s 2s/step - loss: 0.3582 - accuracy: 0.8526 - val_loss: 0.4230 - val_accuracy: 0.8167
Epoch 3/4
 13/373 [>.............................] - ETA: 9:17 - loss: 0.2126 - accuracy: 0.9135

In [ ]:
# Prediction by BERT model with my tuning
model.load_weights('model.h5')
y_pred = model.predict(X_test)
y_pred = y_pred.round().astype('int')

In [ ]:
from sklearn.metrics import accuracy_score

ans = pd.read_csv(f'{url}/ans.csv')['target'].values
accuracy_score(y_pred= y_pred, y_true= ans)

In [ ]:
submission = pd.DataFrame()
submission['id'] = test_df['id']
submission['target'] = y_pred.reshape(-1)
submission.to_csv(f'{url}/submission_BERT.csv', index = False)